# Merit-Based Fairness Metrics

**Category Focus:** *"Protecting qualified individuals"*

This notebook explores Merit-Based Fairness Metrics that ensure qualified people get fair opportunities and innocent people aren't wrongly penalized. These metrics focus on **fair outcomes** based on actual merit, making them essential for high-stakes decisions like hiring, admissions, and promotions.

## Metrics in This Category

1. **Equal Opportunity** - Qualified candidates get equal chances
2. **Average Odds** - Metric summarizing TPR/FPR differences across groups (used to assess how close you are to Equalized Odds)
3. **Predictive Equality** - Ensures that innocent individuals have equal protection from false approvals across all demographic groups

---

## 📚 Descriptive Analysis

### Equal Opportunity
**Definition:** Ensures that individuals who deserve a positive outcome have an equal chance of receiving it across demographic groups.

**Formula:** TPR (True Positive Rate) should be equal across groups
- TPR_group1 ≈ TPR_group2
- Where TPR = True Positives / (True Positives + False Negatives)

**Business Meaning:** 
- Qualified candidates from all demographic groups have equal chances of being selected
- Prevents systematic exclusion of qualified talent from underrepresented groups
- Critical for merit-based hiring and promotion decisions

**When to Use:**
- Hiring processes where missing qualified candidates is costly
- College admissions to ensure qualified students aren't overlooked
- Promotion decisions to maintain talent pipeline equity

---

### Average Odds
**Definition:** A metric that measures how far a model is from satisfying Equalized Odds by calculating the average difference in TPR and FPR across demographic groups.

**Formula:** Average of TPR and FPR differences across groups
- Average Odds = (|TPR_group1 - TPR_group2| + |FPR_group1 - FPR_group2|) / 2
- Where TPR = True Positive Rate and FPR = False Positive Rate

**Relationship to Equalized Odds:**
- **Average Odds** is a *metric* that quantifies unfairness
- **Equalized Odds** is a *fairness constraint* (the goal: TPR and FPR equal across groups)
- Lower Average Odds scores indicate closer proximity to achieving Equalized Odds

**Business Meaning:**
- Measures comprehensive fairness for both qualified candidates and those who don't meet criteria
- Assesses whether the model treats both groups equally in terms of true positives (qualified) and false positives (unqualified)
- Provides a single score to track progress toward the Equalized Odds fairness goal

**When to Use:**
- High-stakes decisions with dual concerns (hiring + avoiding poor hires)
- Credit lending (protect good borrowers + avoid bad loans)
- Medical diagnosis (catch diseases + avoid false alarms)

---

### Predictive Equality
**Definition:** Ensures that individuals who don't deserve a positive outcome have equal protection from false approvals across demographic groups.

**Formula:** FPR (False Positive Rate) should be equal across groups
- FPR_group1 ≈ FPR_group2
- Where FPR = False Positives / (False Positives + True Negatives)

**Business Meaning:**
- Innocent individuals from all demographic groups equally protected from false approvals
- Prevents systematic over-penalization of specific groups
- Focuses on fairness for those who should NOT receive positive outcomes

**When to Use:**
- Criminal justice systems (protect innocent people)
- Fraud detection (avoid falsely flagging legitimate transactions)
- Security screening (minimize false alarms for innocent people)

---

## 🔍 Key Differences

| Metric | Focuses On | Protection Priority | Use Case |
|--------|------------|-------------------|----------|
| **Equal Opportunity** | Qualified individuals only | Missing talent | Talent acquisition |
| **Average Odds** | Both qualified & innocent | Comprehensive fairness measurement | High-stakes decisions |
| **Predictive Equality** | Innocent individuals only | False approvals | Risk/security systems |

---

## 💻 Computational Analysis

Let's implement and compare these three merit-based fairness metrics using the Adult Income dataset.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from jurity.fairness import BinaryFairnessMetrics
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load and prepare the Adult Income dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
          'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
          'hours_per_week', 'native_country', 'income']

data = pd.read_csv(url, names=columns, skipinitialspace=True)

print(f"Dataset shape: {data.shape}")
print("\nFirst few rows:")
print(data.head())
print("\nTarget distribution:")
print(data['income'].value_counts())
print("\nGender distribution:")
print(data['sex'].value_counts())

In [ ]:
# Data preprocessing
# Remove rows with missing values (marked as '?')
data = data.replace('?', np.nan).dropna()

# Prepare features for modeling
# Select relevant features
features = ['age', 'workclass', 'education', 'marital_status', 'occupation', 
           'relationship', 'race', 'capital_gain', 'capital_loss', 'hours_per_week']

X = data[features].copy()
y = (data['income'] == '>50K').astype(int)  # Binary target: 1 for >50K, 0 for <=50K
sensitive_attribute = (data['sex'] == 'Female').astype(int)  # 1 for Female, 0 for Male

# Encode categorical variables
le = LabelEncoder()
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race']
for col in categorical_cols:
    X[col] = le.fit_transform(X[col])

# Reset indices to ensure alignment
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
sensitive_attribute = sensitive_attribute.reset_index(drop=True)

print(f"Processed dataset shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")
print(f"Sensitive attribute distribution: {sensitive_attribute.value_counts().to_dict()}")

In [ ]:
# Split data and train model
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive_attribute, test_size=0.3, random_state=42, stratify=y
)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("Model trained successfully!")
print(f"Test set accuracy: {(y_pred == y_test).mean():.3f}")
print(f"Test set size: {len(y_test)}")

In [ ]:
# Calculate Merit-Based Fairness Metrics using Jurity
bfm = BinaryFairnessMetrics()

# Equal Opportunity
equal_opportunity_score = bfm.EqualOpportunity.get_score(
    labels=y_test.values,
    predictions=y_pred,
    memberships=sensitive_test.values
)

# Average Odds - metric measuring distance from Equalized Odds fairness constraint
average_odds_score = bfm.AverageOdds.get_score(
    labels=y_test.values,
    predictions=y_pred,
    memberships=sensitive_test.values
)

# Predictive Equality
predictive_equality_score = bfm.PredictiveEquality.get_score(
    labels=y_test.values,
    predictions=y_pred,
    memberships=sensitive_test.values
)

print("=== MERIT-BASED FAIRNESS METRICS RESULTS ===")
print(f"Equal Opportunity Score: {equal_opportunity_score:.4f}")
print(f"Average Odds Score: {average_odds_score:.4f}")
print(f"Predictive Equality Score: {predictive_equality_score:.4f}")
print("\nNote: Scores closer to 0 indicate better fairness (smaller differences between groups)")

In [ ]:
# Manual calculation and verification of metrics
def calculate_confusion_matrix_by_group(y_true, y_pred, sensitive):
    """Calculate confusion matrix metrics by sensitive group"""
    results = {}
    
    for group in [0, 1]:  # 0=Male, 1=Female
        mask = sensitive == group
        y_true_group = y_true[mask]
        y_pred_group = y_pred[mask]
        
        # Confusion matrix
        tn = ((y_true_group == 0) & (y_pred_group == 0)).sum()
        tp = ((y_true_group == 1) & (y_pred_group == 1)).sum()
        fn = ((y_true_group == 1) & (y_pred_group == 0)).sum()
        fp = ((y_true_group == 0) & (y_pred_group == 1)).sum()
        
        # Calculate rates
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Positive Rate
        
        group_name = 'Female' if group == 1 else 'Male'
        results[group_name] = {
            'TPR': tpr,
            'FPR': fpr,
            'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn,
            'Total': len(y_true_group)
        }
    
    return results

# Calculate metrics by group
group_metrics = calculate_confusion_matrix_by_group(y_test.values, y_pred, sensitive_test.values)

print("=== DETAILED GROUP PERFORMANCE METRICS ===")
for group, metrics in group_metrics.items():
    print(f"\n{group} Group:")
    print(f"  Total samples: {metrics['Total']}")
    print(f"  True Positive Rate (TPR): {metrics['TPR']:.4f}")
    print(f"  False Positive Rate (FPR): {metrics['FPR']:.4f}")
    print(f"  Confusion Matrix: TP={metrics['TP']}, TN={metrics['TN']}, FP={metrics['FP']}, FN={metrics['FN']}")

In [ ]:
# Manual calculation of merit-based fairness metrics
male_tpr = group_metrics['Male']['TPR']
female_tpr = group_metrics['Female']['TPR']
male_fpr = group_metrics['Male']['FPR']
female_fpr = group_metrics['Female']['FPR']

# Calculate fairness metric differences
equal_opportunity_diff = abs(female_tpr - male_tpr)
predictive_equality_diff = abs(female_fpr - male_fpr)
average_odds_diff = (equal_opportunity_diff + predictive_equality_diff) / 2

print("=== MANUAL VERIFICATION OF FAIRNESS METRICS ===")
print(f"\nEqual Opportunity (TPR difference):")
print(f"  Male TPR: {male_tpr:.4f}")
print(f"  Female TPR: {female_tpr:.4f}")
print(f"  Difference: {equal_opportunity_diff:.4f}")
print(f"  Jurity Score: {equal_opportunity_score:.4f}")
print(f"  ✓ Match: {abs(equal_opportunity_diff - equal_opportunity_score) < 0.001}")

print(f"\nPredictive Equality (FPR difference):")
print(f"  Male FPR: {male_fpr:.4f}")
print(f"  Female FPR: {female_fpr:.4f}")
print(f"  Difference: {predictive_equality_diff:.4f}")
print(f"  Jurity Score: {predictive_equality_score:.4f}")
print(f"  ✓ Match: {abs(predictive_equality_diff - predictive_equality_score) < 0.001}")

print(f"\nAverage Odds (Average of TPR and FPR differences):")
print(f"  Calculated Average: {average_odds_diff:.4f}")
print(f"  Jurity Score: {average_odds_score:.4f}")
print(f"  ✓ Match: {abs(average_odds_diff - average_odds_score) < 0.001}")

In [ ]:
# Create comprehensive visualization dashboard
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Merit-Based Fairness Metrics Analysis Dashboard', fontsize=16, fontweight='bold')

# 1. Group Performance Comparison
groups = ['Male', 'Female']
tpr_values = [male_tpr, female_tpr]
fpr_values = [male_fpr, female_fpr]

x = np.arange(len(groups))
width = 0.35

axes[0,0].bar(x - width/2, tpr_values, width, label='TPR (True Positive Rate)', alpha=0.8)
axes[0,0].bar(x + width/2, fpr_values, width, label='FPR (False Positive Rate)', alpha=0.8)
axes[0,0].set_xlabel('Demographic Group')
axes[0,0].set_ylabel('Rate')
axes[0,0].set_title('Performance Rates by Group')
axes[0,0].set_xticks(x)
axes[0,0].set_xticklabels(groups)
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Equal Opportunity Visualization
axes[0,1].bar(groups, tpr_values, color=['skyblue', 'lightcoral'], alpha=0.8)
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].set_title(f'Equal Opportunity\n(TPR Difference: {equal_opportunity_diff:.4f})')
axes[0,1].grid(True, alpha=0.3)
for i, v in enumerate(tpr_values):
    axes[0,1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# 3. Predictive Equality Visualization
axes[0,2].bar(groups, fpr_values, color=['lightgreen', 'orange'], alpha=0.8)
axes[0,2].set_ylabel('False Positive Rate')
axes[0,2].set_title(f'Predictive Equality\n(FPR Difference: {predictive_equality_diff:.4f})')
axes[0,2].grid(True, alpha=0.3)
for i, v in enumerate(fpr_values):
    axes[0,2].text(i, v + 0.005, f'{v:.3f}', ha='center', va='bottom')

# 4. Fairness Scores Comparison
fairness_scores = [equal_opportunity_score, predictive_equality_score, average_odds_score]
fairness_names = ['Equal\nOpportunity', 'Predictive\nEquality', 'Average\nOdds']
colors = ['skyblue', 'lightgreen', 'plum']

bars = axes[1,0].bar(fairness_names, fairness_scores, color=colors, alpha=0.8)
axes[1,0].set_ylabel('Fairness Score')
axes[1,0].set_title('Merit-Based Fairness Scores\n(Lower = More Fair)')
axes[1,0].grid(True, alpha=0.3)
for bar, score in zip(bars, fairness_scores):
    axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
                  f'{score:.4f}', ha='center', va='bottom')

# 5. Confusion Matrix Heatmaps
for idx, (group, metrics) in enumerate(group_metrics.items()):
    cm = np.array([[metrics['TN'], metrics['FP']], 
                   [metrics['FN'], metrics['TP']]])
    
    im = axes[1, idx+1].imshow(cm, cmap='Blues', alpha=0.8)
    axes[1, idx+1].set_title(f'{group} Group\nConfusion Matrix')
    
    # Add text annotations
    for i in range(2):
        for j in range(2):
            text = axes[1, idx+1].text(j, i, cm[i, j], ha="center", va="center", color="black", fontweight='bold')
    
    axes[1, idx+1].set_xticks([0, 1])
    axes[1, idx+1].set_yticks([0, 1])
    axes[1, idx+1].set_xticklabels(['Predicted: ≤50K', 'Predicted: >50K'])
    axes[1, idx+1].set_yticklabels(['Actual: ≤50K', 'Actual: >50K'])

# 6. Business Risk Assessment
def assess_merit_fairness_risk(eo_score, pe_score, ao_score):
    """Assess business risk based on merit-based fairness scores"""
    risks = []
    
    if eo_score > 0.1:
        risks.append(f"HIGH RISK: Equal Opportunity gap ({eo_score:.3f}) - Losing qualified talent")
    elif eo_score > 0.05:
        risks.append(f"MEDIUM RISK: Equal Opportunity gap ({eo_score:.3f}) - Monitor talent pipeline")
    else:
        risks.append(f"LOW RISK: Equal Opportunity gap ({eo_score:.3f}) - Good talent fairness")
    
    if pe_score > 0.1:
        risks.append(f"HIGH RISK: Predictive Equality gap ({pe_score:.3f}) - Unfair false approvals")
    elif pe_score > 0.05:
        risks.append(f"MEDIUM RISK: Predictive Equality gap ({pe_score:.3f}) - Monitor false positives")
    else:
        risks.append(f"LOW RISK: Predictive Equality gap ({pe_score:.3f}) - Good false positive fairness")
    
    if ao_score > 0.1:
        risks.append(f"HIGH RISK: Average Odds gap ({ao_score:.3f}) - Far from Equalized Odds goal")
    elif ao_score > 0.05:
        risks.append(f"MEDIUM RISK: Average Odds gap ({ao_score:.3f}) - Progress needed toward Equalized Odds")
    else:
        risks.append(f"LOW RISK: Average Odds gap ({ao_score:.3f}) - Close to Equalized Odds goal")
    
    return risks

risk_assessment = assess_merit_fairness_risk(equal_opportunity_score, predictive_equality_score, average_odds_score)

# 7. Risk Assessment Text
axes[2,0].axis('off')
risk_text = "BUSINESS RISK ASSESSMENT:\n\n" + "\n\n".join(risk_assessment)
axes[2,0].text(0.05, 0.95, risk_text, transform=axes[2,0].transAxes, 
              fontsize=10, verticalalignment='top', fontfamily='monospace',
              bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

# 8. Metric Interpretation Guide
axes[2,1].axis('off')
interpretation_text = (
    "METRIC INTERPRETATION:\n\n"
    "Equal Opportunity:\n"
    "• Focuses on qualified individuals\n"
    "• Measures TPR gap between groups\n"
    "• Critical for talent acquisition\n\n"
    "Predictive Equality:\n"
    "• Focuses on innocent individuals\n"
    "• Measures FPR gap between groups\n"
    "• Critical for false accusation protection\n\n"
    "Average Odds:\n"
    "• Metric measuring distance from\n"
    "  Equalized Odds (fairness goal)\n"
    "• Combines TPR and FPR gaps\n"
    "• Best for tracking comprehensive fairness"
)
axes[2,1].text(0.05, 0.95, interpretation_text, transform=axes[2,1].transAxes,
              fontsize=9, verticalalignment='top', fontfamily='monospace',
              bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow", alpha=0.8))

# 9. Recommendations
axes[2,2].axis('off')
recommendations = (
    "ACTIONABLE RECOMMENDATIONS:\n\n"
    f"1. TALENT PIPELINE:\n"
    f"   TPR Gap: {equal_opportunity_diff:.3f}\n"
    f"   {'✓ Acceptable' if equal_opportunity_diff < 0.05 else '⚠ Review recruitment'}\n\n"
    f"2. FALSE POSITIVE PROTECTION:\n"
    f"   FPR Gap: {predictive_equality_diff:.3f}\n"
    f"   {'✓ Fair protection' if predictive_equality_diff < 0.05 else '⚠ Review thresholds'}\n\n"
    f"3. EQUALIZED ODDS PROGRESS:\n"
    f"   Average Odds: {average_odds_diff:.3f}\n"
    f"   {'✓ Close to goal' if average_odds_diff < 0.05 else '⚠ Needs improvement'}\n\n"
    "Next Steps:\n"
    "• Monitor metrics regularly\n"
    "• Audit decision thresholds\n"
    "• Review training data balance"
)
axes[2,2].text(0.05, 0.95, recommendations, transform=axes[2,2].transAxes,
              fontsize=9, verticalalignment='top', fontfamily='monospace',
              bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))

plt.tight_layout()
plt.show()

## 🎯 Business Impact Analysis

### Key Findings from Our Analysis

Based on the computational results above, we can assess the merit-based fairness of our income prediction model:

#### Equal Opportunity Analysis
- **Metric Focus:** Ensures qualified candidates from both genders have equal chances of being predicted as high earners
- **Business Impact:** Critical for talent retention and equal opportunity in hiring/promotion decisions
- **Risk Level:** Determined by TPR difference between demographic groups

#### Predictive Equality Analysis
- **Metric Focus:** Ensures individuals who shouldn't be predicted as high earners have equal protection from false positives
- **Business Impact:** Prevents unfair advantages or false elevation of unqualified candidates
- **Risk Level:** Determined by FPR difference between demographic groups

#### Average Odds Analysis
- **Metric Focus:** Quantifies how far the model is from achieving Equalized Odds (the fairness constraint where both TPR and FPR are equal across groups)
- **Business Impact:** Provides a single comprehensive metric to track progress toward balanced fairness for both qualified and unqualified individuals
- **Risk Level:** Combined assessment of both TPR and FPR fairness gaps
- **Note:** Average Odds is a *measurement tool*, while Equalized Odds is the *fairness goal* we aim to achieve

---

### When to Use Each Metric

**Use Equal Opportunity when:**
- Primary concern is not missing qualified talent
- Hiring for competitive positions where qualifications matter most
- Cost of false negatives (missing good candidates) is high

**Use Predictive Equality when:**
- Primary concern is avoiding false approvals or unfair treatment
- Security or risk assessment applications
- Cost of false positives (wrongly selecting unqualified) is high

**Use Average Odds when:**
- Need to measure comprehensive fairness across both concerns
- Want a single metric to track progress toward Equalized Odds
- High-stakes decisions with broad impact requiring balanced fairness
- Regulatory requirements for demonstrating fairness improvements

---

## 💼 Stakeholder Implications

### For HR and Talent Teams
Merit-based fairness metrics directly impact talent management:
- **Equal Opportunity** ensures no qualified talent is systematically overlooked
- **Predictive Equality** prevents unfair advantages for unqualified candidates
- **Average Odds** provides a comprehensive score to track overall fairness progress toward Equalized Odds

### For Risk Management
These metrics help assess and mitigate business risks:
- Talent pipeline risks (losing qualified candidates)
- Quality risks (selecting unqualified candidates)
- Legal risks (discrimination in merit-based decisions)
- Reputational risks (perceived unfairness in high-stakes decisions)

### For Executive Leadership
Merit-based fairness metrics support strategic objectives:
- Ensure competitive advantage through fair talent acquisition
- Maintain organizational reputation for fairness
- Meet regulatory requirements while optimizing for merit
- Balance fairness with business performance objectives
- Track progress toward fairness goals using Average Odds as a comprehensive KPI
